In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
def detect_sign(yolo_path, frame):
    # 加载模型
    yolo = YOLO(yolo_path)
    
    # 读取图片
    img = cv2.imread(frame) if isinstance(frame, str) else frame
    
    # 进行预测
    results = yolo.predict(img)[0]  # 只获取第一个结果
    
    # 在图片上绘制每个检测框
    for box in results.boxes.data:
        # 获取坐标和置信度
        x1, y1, x2, y2, conf, cls = box
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # 转换为整数
        
        # 绘制边界框
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # 获取类别名称
        cls_name = results.names[int(cls)]
        
        # 添加标签文本
        label = f'{cls_name} '
        cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, (0, 255, 0), 2)
    
    return img

# img = detect_sign("./runs/detect/train45/weights/best.pt", "./datasets/datasets_hw/images/train/frame55.jpg")
# cv2.imshow("Detection", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [5]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt

def detect_line(image):
    # 读取和处理图像的代码保持不变...
    #image = cv2.imread(image_path)
    height, width = image.shape[:2]
    
    # 定义感兴趣区域的顶点
    # 这里我们创建一个梯形区域，这个区域通常包含了车道线的主要部分
    roi_vertices = np.array([
        # 左下角
        [width * 0, height],
        # 左上角
        [width * 0, height * 0.25],
        # 右上角
        [width * 0.9, height * 0.25],
        # 右下角
        [width * 1, height]
    ], dtype=np.int32)
     # 创建ROI掩码
    roi_mask = np.zeros_like(image)
    cv2.fillPoly(roi_mask, [roi_vertices], (255, 255, 255))
    
    # 1. 颜色检测部分
    # 转换到HSV空间
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # 白色阈值（调整饱和度范围来处理反光）
    white_lower = np.array([0, 0, 200])
    white_upper = np.array([180, 40, 240])
    
    # 黄色阈值
    yellow_lower = np.array([15, 80, 80])
    yellow_upper = np.array([45, 255, 255])
    
    # 创建颜色掩码
    white_mask = cv2.inRange(hsv, white_lower, white_upper)
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)
    color_mask = cv2.bitwise_or(white_mask, yellow_mask)
    
    # 2. 边缘检测部分
    # 转换到灰度图
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 使用高斯模糊减少噪声
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    
    # Canny边缘检测
    # 使用较低的阈值来检测更多的边缘
    edges = cv2.Canny(blurred, 50, 150)  # More selective thresholds
    
    # 3. 组合颜色检测和边缘检测的结果
    # 使用形态学操作来加强边缘连接
    kernel = np.ones((3,3), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)
    

    # 计算边缘强度图
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    edge_magnitude = np.sqrt(sobelx**2 + sobely**2)
    
    # 设定边缘强度阈值
    edge_threshold = np.mean(edge_magnitude) * 1.5
    strong_edges = (edge_magnitude > edge_threshold).astype(np.uint8) * 255
    
    # 将强边缘与 Canny 边缘结合
    final_edges = cv2.bitwise_and(edges, strong_edges)
        
    #将颜色掩码和边缘检测结果相与
    combined_mask = cv2.bitwise_and(color_mask,final_edges)
    
    # 应用ROI掩码
    final_mask = cv2.bitwise_and(combined_mask, roi_mask[:,:,0])
    # 然后再进行霍夫变换
    lines = cv2.HoughLinesP(
        final_mask,
        rho=1,
        theta=np.pi/180,
        threshold=50,
        minLineLength=25,
        maxLineGap=40
    )
        
    # 创建结果图像
    result = image.copy()
    
    # 在绘制检测到的直线之前添加以下代码
    # if lines is not None:
    #     filtered_lines = []
    #     for line in lines:
    #         x1, y1, x2, y2 = line[0]
    #         # 计算线段角度
    #         angle = abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
    #         
    #         # 筛选接近垂直的线条（可以调整角度范围）
    #         # 车道线通常在20-70度之间
    #         if (20 <= angle <= 70) or (110 <= angle <= 160):
    #             filtered_lines.append(line)
    #     
    #     # 使用筛选后的线条绘制
    #     for line in filtered_lines:
    #         x1, y1, x2, y2 = line[0]
    #         cv2.line(result, (x1, y1), (x2, y2), (0, 0, 255), 2)
        
    # 绘制检测到的直线
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            #print(line)
            cv2.line(result, (x1, y1), (x2, y2), (0, 0, 255), 1)
    if result is None:
        print("警告：生成的结果图像为空")
        return None
        
    # 确保图像是uint8类型
    if result.dtype != np.uint8:
        result = result.astype(np.uint8)
        
    # 确保图像是BGR格式（OpenCV需要的格式）
    if len(result.shape) != 3 or result.shape[2] != 3:
        print("警告：输出图像格式不正确")
        return None
        
    # 在返回之前进行验证
    try:
        # 测试是否可以编码为视频帧
        _, test_encode = cv2.imencode('.jpg', result)
        if not test_encode.any():
            print("警告：图像编码测试失败")
            return None
    except Exception as e:
        print(f"警告：图像处理出错 - {e}")
        return None

    # # 显示处理过程
    # plt.figure(figsize=(15, 10))
    # 
    # plt.subplot(2, 3, 1)
    # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # plt.title('原始图像')
    # 
    # plt.subplot(2, 3, 2)
    # plt.imshow(color_mask, cmap='gray')
    # plt.title('颜色检测')
    # 
    # plt.subplot(2, 3, 3)
    # plt.imshow(edges, cmap='gray')
    # plt.title('Canny边缘检测')
    # 
    # plt.subplot(2, 3, 4)
    # plt.imshow(final_mask, cmap='gray')
    # plt.title('组合结果')
    # 
    # plt.subplot(2, 3, 5)
    # plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    # plt.title('最终检测结果')
    # 
    # # 显示带ROI的结果
    # debug_image = result.copy()
    # cv2.polylines(debug_image, [roi_vertices], True, (0, 255, 0), 2)
    # 
    # plt.subplot(2, 3, 6)
    # plt.imshow(cv2.cvtColor(debug_image, cv2.COLOR_BGR2RGB))
    # plt.title('带ROI的结果')
    # 
    # plt.tight_layout()
    # plt.show()

    return result


In [6]:
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    count = 0
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            print(f"Processing frame: {count}")
            count += 1
            
            # 处理帧
            pr_frame = detect_line(frame)
            pr_frame = detect_sign("./runs/detect/train45/weights/best.pt", pr_frame)
            
            if pr_frame is not None:
                out.write(pr_frame)
                
    finally:
        # 确保资源被正确释放
        cap.release()
        out.release()
        cv2.destroyAllWindows()

In [8]:
process_video("./video.mp4", "./final.mp4")

Processing frame: 0

0: 384x640 (no detections), 12.5ms
Speed: 13.2ms preprocess, 12.5ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame: 1

0: 384x640 (no detections), 6.2ms
Speed: 1.3ms preprocess, 6.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Processing frame: 2

0: 384x640 (no detections), 9.2ms
Speed: 1.2ms preprocess, 9.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Processing frame: 3

0: 384x640 (no detections), 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame: 4

0: 384x640 (no detections), 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame: 5

0: 384x640 (no detections), 6.3ms
Speed: 1.5ms preprocess, 6.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Processing frame: 6

0: 384x640 (no detections), 7.7ms
Speed: 1.0ms preprocess, 7.7ms infer